In [106]:
import pandas as pd
import os
import re

In [115]:
vanilla_pref = "./surprisal/vanilla/"
vanilla = [vanilla_pref + f for f in os.listdir(vanilla_pref) if not f.startswith(".")]
rnng_pref = "./surprisal/rnng/"
rnng = [rnng_pref + f for f in os.listdir(rnng_pref) if not f.startswith(".")]
on_pref = "./surprisal/ordered-neurons/"
ordered_n = [on_pref + f for f in os.listdir(on_pref) if not f.startswith(".")]

results = vanilla + rnng + ordered_n

natural_ref = pd.read_csv("natural_reference.csv")
natural_ref = natural_ref.drop(["Unnamed: 0"], axis=1)
natural_ref["code"] = natural_ref["zone"].map(str) + "_" + natural_ref["item"].map(str)
natural_ref = natural_ref.drop(["zone", "item"], axis=1)

brown_ref = pd.read_csv("brown_reference.csv")
brown_ref = brown_ref.drop(["Unnamed: 0", "subject", "text_id", "text_pos", "word_in_exp", "time"], axis=1)
brown_ref.head()

result = []

for res in results:
    tag = tuple(res.split("/")[3].replace(".csv", "").split("_"))
    d = [tuple(x) for x in pd.read_csv(res, sep="\t").values.tolist()]
    if ("bnc-brown" in res):
        ref = [tuple(x) for x in brown_ref.values.tolist()]
    else:
        ref = [tuple(x) for x in natural_ref.values.tolist()]
        
    curr_d = d.pop(0)
    curr_ref = ref.pop(0)
    
    while len(d) > 10:
        #print(curr_d[2] + " " + curr_ref[0])
        if curr_d[2] == curr_ref[0]:
            #print("===" + curr_d[2] + "-" + curr_ref[0])
            result.append(curr_d + curr_ref + tag)
            curr_d = d.pop(0)
            curr_ref = ref.pop(0)
        # If current token is unked, then pop both
        elif "UNK" in curr_d[2]:
            curr_d = d.pop(0)
            curr_ref = ref.pop(0)
        # If current ref has punctuation, pop both
        elif not curr_ref[0].isalpha():
            curr_ref = ref.pop(0)
            curr_d = d.pop(0)
        else:
            curr_d = d.pop(0)
        
print(len(result))
res = pd.DataFrame(result)
res.columns = ["x1", "x2", "word", "surprisal", "word2", "code", "corpus", "model", "training", "seed"]
res.drop(["x1", "x2", "word2"], axis = 1)
res.to_csv("coded_results.csv")

    

170486


In [111]:
res.head()

,0,1,2,3,4,5,6,7,8,9
0,1,1,If,0.000000,If,1_1,natural-stories,vanilla,bllip-md,0922
1,1,1,you,3.097762,you,2_1,natural-stories,vanilla,bllip-md,0922
2,1,2,were,3.480068,were,3_1,natural-stories,vanilla,bllip-md,0922
3,1,3,to,3.205047,to,4_1,natural-stories,vanilla,bllip-md,0922
4,1,4,journey,14.515004,journey,5_1,natural-stories,vanilla,bllip-md,0922
